In [1]:
#Importing modules
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions
import pandas as pd

options = ChromeOptions() #make headless browser
options.headless = True #make headless browser

#calling the driver I downloaded at this file location
driver_path="C:/Users/drews/Documents/code_practice/chromedriver_win32/chromedriver"

driver = Chrome(executable_path=driver_path, options=options)

#accessing the website through chrome
driver.get('https://www.economicmodeling.com/open-positions/')

#The additional information was easier to get with the tags, as they all came together
#the info list contains Department, Location and Job Type for each element
job_title_list=[]
info_list=[]
link_list=[]

#making a function to save each element gathered into a list of strings
def Lists_fn(var,var_list):
    for element in var:
        i=element.text
        var_list.append(str(i))


#Scraped the title by the class name
job_title = driver.find_elements_by_class_name("job-title")

#The additional information was easier to get with the tags, as they all came together
info = driver.find_elements_by_class_name("tags")

#get links from job_title attributes and save in list
for lnk in job_title:
    j=lnk.get_attribute('href')
    link_list.append(str(j))


#Calling the funtion to save the scrapped lists as text in new lists
Lists_fn(job_title,job_title_list)
Lists_fn(info,info_list)

driver.quit()



C:\Users\drews\anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  if sys.path[0] == '':
C:\Users\drews\anaconda3\lib\site-packages\ipykernel_launcher.py:31: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
C:\Users\drews\anaconda3\lib\site-packages\ipykernel_launcher.py:34: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead


In [2]:
#making a new driver because the first one was stale
#options=options to make this a headless browser
driver2 = Chrome(executable_path = driver_path, options=options)

#Making an empty list for the job descriptions I plan to gether from each link
job_description_list=[]

#getting job descriptions from the list of links

#for each link in the list the loop will acess that link, and gather the job description from
#the xpath, and save the description into a list
for n in link_list:
    
    #Launch second driver
    driver2.get(n)

    #scraping the job descriptions from the xpath
    job_description = driver2.find_element_by_xpath("/html/body/div[3]/div/div[2]")
    
    #saving each job description to a list as a string
    job_description_list.append(job_description.text)


C:\Users\drews\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\drews\anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead


In [3]:
#I now have each item in seperate iterable lists
#I decided the best way to store this information would 
#be to use a pandas DataFrame


#Zipping the lists to make a dataframe
zipped = list(zip(job_title_list,info_list,job_description_list,link_list))

emsi_bg_jobs_df=pd.DataFrame(zipped, columns=['Job_Title','Info','Job_Description','Link'])
#splitting the Info into its seperate parts
emsi_bg_jobs_df[['Department','Location','Type']] = emsi_bg_jobs_df.Info.str.split('\n',expand=True)
#Remove redundant column
emsi_bg_jobs_df.drop("Info", axis=1, inplace=True)

#reorder columns for clarity
column_names = ['Job_Title','Job_Description','Location','Type','Department','Link']
emsi_bg_jobs_df = emsi_bg_jobs_df.reindex(columns=column_names)

#displaying the first 5 rows of the DataFrame
display(emsi_bg_jobs_df.head())

#saving the DataFrame in a json file
emsi_bg_jobs_df.to_json("Emsi_BG_Jobs.json", orient = "index")

##to open json file
# import json
# data = json.load (open ("Emsi_BG_Jobs.json", "r")) 
# opened_df = pd.DataFrame.from_dict (data, orient="index")
# display(opened_df.head())

,Job_Title,Job_Description,Location,Type,Department,Link
0,Education Success Team Lead,"We are seeking a collaborative, detail-oriente...","Moscow, ID",Full Time,Client Services,https://jobs.lever.co/economicmodeling/390d26a...
1,Data Curator,A Data Curator at Emsi BG is responsible for e...,"Moscow, ID",Full Time,Data and Analytics,https://jobs.lever.co/economicmodeling/d5ff570...
2,"Data Engineer, Web Scraping",Emsi Burning Glass is looking for a talented D...,Remote (Work From Home) US,Full Time,Data and Analytics,https://jobs.lever.co/economicmodeling/1411c67...
3,Data Scientist,A Data Scientist at Emsi Burning Glass works w...,Remote (Work From Home) US,Full Time,Data and Analytics,https://jobs.lever.co/economicmodeling/e846f31...
4,Director of Data Analytics,The Director of Data Analytics leads data scie...,"Boston, Massachusetts",Full Time,Data and Analytics,https://jobs.lever.co/economicmodeling/932d576...
